<a href="https://colab.research.google.com/github/Stravanni/Basi_di_dati/blob/main/06_SQL_sqlite_RECURSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduzione a SQL
@author: giovanni.simonini@unimore.it

## To run: 
- to run a cell: SHIFT + ENTER

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite://', echo=False)

## Create the the tables

In [3]:
q = '''WITH RECURSIVE cnt(x) AS 
        (SELECT 1 UNION ALL
          SELECT x+1
          FROM cnt 
          WHERE x<1000000)
        SELECT x 
        FROM cnt;'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,x
0,1
1,2
2,3
3,4
4,5


## UNION ALL
- `UNION ALL` **non** rimuove i duplicati, semplicemente fa l'unione delle query
- In questa query il query optimizer capisce che ogni numero generato è utilizzato solo una volta, quindi libera la memoria del numero ad ogni terazione

In [4]:
q = '''WITH RECURSIVE cnt(x) AS 
        (VALUES(1) UNION ALL
          SELECT x+1 
          FROM cnt 
          WHERE x<1000000*x)
        SELECT x 
        FROM cnt
        LIMIT 10;'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.tail()

,x
5,6
6,7
7,8
8,9
9,10


### LIMIT CTE
Un buon metodo per assicurarsi che la qeury finisca è quello di mettere un `LIMIT` per testare che la ricorsione termini.

## Example

In [5]:
q = '''DROP TABLE IF EXISTS employee;'''
res = engine.execute(q)

In [6]:
q = '''CREATE TABLE employee(
        id INT PRIMARY KEY,
        name TEXT NOT NULL,
        manager_id INT REFERENCES employee(id)
        )'''
res = engine.execute(q)

In [7]:
q = '''
INSERT INTO employee (id, name, manager_id)
VALUES 
(1, 'kim', NULL),
(2, 'tom', 1),
(3, 'sam', 1),
(4, 'yoan', NULL),
(5, 'jon', 2),
(6, 'pier', 2),
(7, 'zoe', 4),
(8, 'ian', 1),
(9, 'loic', 4);
'''
res = engine.execute(q)

In [8]:
q = '''SELECT * FROM employee'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,id,name,manager_id
0,1,kim,NaN
1,2,tom,1.0
2,3,sam,1.0
3,4,yoan,NaN
4,5,jon,2.0
5,6,pier,2.0
6,7,zoe,4.0
7,8,ian,1.0
8,9,loic,4.0


## RIPORTI INDIRETTI
- selezioniamo tutti i riporti (diretti e indiretti) di Kim

In [9]:
q = '''
WITH RECURSIVE cte_employee (id, name, manager_id) AS (
  SELECT e.id, e.name, e.manager_id
  FROM employee e
  WHERE e.id = 1
    UNION ALL
  SELECT e.id, e.name, e.manager_id
  FROM employee e
  JOIN cte_employee c ON c.id = e.manager_id
)

SELECT * FROM cte_employee;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,id,name,manager_id
0,1,kim,NaN
1,2,tom,1.0
2,3,sam,1.0
3,8,ian,1.0
4,5,jon,2.0
5,6,pier,2.0


### SPIEGAZIONE
- il `SELECT` iniziale è eseguito e ritorna (1, kim), mettendolo in una coda *content table*
- (1, kim) è rimosso dalla coda e viene eseguito ricorsivamente la nuova `SELECT` (dopo lo `UNION ALL`) considerando che in  cte_employee per ora ci sia solo (1, kim)
- vengono così ritornati (2, tom), (3, sam), (8, ian), cioè i riporti di Kim, e vengono messi nella coda *content table*
  - `tom`, `ian` e `sam` sono infatti i record della tabella `employee` per cui c'è un join tra `manater_id` e `id` della cte_employee (che per ora contiene solo `kim`)
- a questo punto viene rimosso dalla coda (2, tom), e di ancora una volra viene eseguita ricorsivamente la query, con join come al punto precedente
  - qeusta volta vengono messi in coda `jon` e `pier`
- ora viene tolto dalla coda `sam` e come prima viene eseguita la query ricorsiva 
  - `sam` non ha riporti, quindi non c'è nessun join questa volta
  - niente viene aggiunto alla coda
- viene quindi tolto dalla coda (8, ian), poi (5, jon) e infine (6, pier)
- quando la coda è vuota, viene ritornato tutto il valore della tabella